In [1]:
# imports
import datasetdatabase as dsdb
import pandas as pd
import numpy as np
import pathlib
import quilt
import json
import os

In [2]:
# connection manager
mngr = dsdb.ConnectionManager(dsdb.LOCAL, user="jacksonb")
mngr

LOCAL:
	driver: sqlite
	link: /active/examples/local_database/local.db

In [3]:
# database manager
local = mngr.connect(dsdb.LOCAL)
local

Recent Datasets:
--------------------------------------------------------------------------------

In [4]:
# create test upload dataset
fp_ex = pathlib.Path("/active/examples/fp_example/")
if not fp_ex.exists():
    os.makedirs(fp_ex)

np.random.seed(seed=12)

test = []
for i in range(10):
    fp =  fp_ex / (str(i) + ".json")
    with open(fp, "w") as write_out:
        json.dump({"hello": "world"}, write_out)
    
    d = {}
    d["strings"] = "foo" + str(i)
    d["bools"] = np.random.rand() < 0.5
    d["floats"] = np.random.rand() * 100
    d["ndarrays"] = np.random.rand(2, 2)
    d["tuples"] = tuple([1, 2, 3])
    d["sets"] = set([1, 2, 3, 3, 3])
    d["files"] = str(fp)
    test.append(d)

test = pd.DataFrame(test)
test.to_csv(fp_ex / "example.csv")
test

,bools,files,floats,ndarrays,sets,strings,tuples
0,True,/active/examples/fp_example/0.json,74.004970,"[[0.26331501518513467, 0.5337393933802977], [0...","{1, 2, 3}",foo0,"(1, 2, 3)"
1,False,/active/examples/fp_example/1.json,3.342143,"[[0.9569493362751168, 0.13720932135607644], [0...","{1, 2, 3}",foo1,"(1, 2, 3)"
2,False,/active/examples/fp_example/2.json,85.273554,"[[0.002259233518513537, 0.5212260272202929], [...","{1, 2, 3}",foo2,"(1, 2, 3)"
3,False,/active/examples/fp_example/3.json,16.071675,"[[0.7645604503388788, 0.020809797952066167], [...","{1, 2, 3}",foo3,"(1, 2, 3)"
4,True,/active/examples/fp_example/4.json,67.145265,"[[0.4712297782500141, 0.8161682980460269], [0....","{1, 2, 3}",foo4,"(1, 2, 3)"
5,False,/active/examples/fp_example/5.json,32.756948,"[[0.3346475291060558, 0.9780580790165189], [0....","{1, 2, 3}",foo5,"(1, 2, 3)"
6,False,/active/examples/fp_example/6.json,82.500925,"[[0.40664030180666166, 0.4513084114213143], [0...","{1, 2, 3}",foo6,"(1, 2, 3)"
7,True,/active/examples/fp_example/7.json,96.259690,"[[0.4192502702591062, 0.4240524465509987], [0....","{1, 2, 3}",foo7,"(1, 2, 3)"
8,True,/active/examples/fp_example/8.json,3.516826,"[[0.08427266973184566, 0.7325206981419501], [0...","{1, 2, 3}",foo8,"(1, 2, 3)"
9,True,/active/examples/fp_example/9.json,22.085252,"[[0.055019993340200135, 0.5232460707782919], [...","{1, 2, 3}",foo9,"(1, 2, 3)"


In [5]:
# upload and return the dataset info
# first upload
ds_info = local.upload_dataset(dataset=test,
                               name="test_dataset",
                               description="this is the hello world of dataset ingestion",
                               type_map={"bools": bool,
                                         "files": str,
                                         "floats": float,
                                         "ndarrays": np.ndarray,
                                         "strings": str},
                               filepath_columns=["files"])
ds_info

{'DatasetId': 1,
 'Name': 'test_dataset',
 'Description': 'this is the hello world of dataset ingestion',
 'SourceId': 1,
 'Created': '2018-07-13 21:01:54.402116'}

In [6]:
# recent datasets
local

Recent Datasets:
--------------------------------------------------------------------------------

In [11]:
local

Recent Datasets:
--------------------------------------------------------------------------------

In [12]:
local.get_dataset(1)

,bools,files,floats,ndarrays,sets,strings,tuples
0,True,/active/examples/fp_example/0.json,74.004970,"[[0.26331501518513467, 0.5337393933802977], [0...","{1, 2, 3}",foo0,"(1, 2, 3)"
1,False,/active/examples/fp_example/1.json,3.342143,"[[0.9569493362751168, 0.13720932135607644], [0...","{1, 2, 3}",foo1,"(1, 2, 3)"
2,False,/active/examples/fp_example/2.json,85.273554,"[[0.002259233518513537, 0.5212260272202929], [...","{1, 2, 3}",foo2,"(1, 2, 3)"
3,False,/active/examples/fp_example/3.json,16.071675,"[[0.7645604503388788, 0.020809797952066167], [...","{1, 2, 3}",foo3,"(1, 2, 3)"
4,True,/active/examples/fp_example/4.json,67.145265,"[[0.4712297782500141, 0.8161682980460269], [0....","{1, 2, 3}",foo4,"(1, 2, 3)"
5,False,/active/examples/fp_example/5.json,32.756948,"[[0.3346475291060558, 0.9780580790165189], [0....","{1, 2, 3}",foo5,"(1, 2, 3)"
6,False,/active/examples/fp_example/6.json,82.500925,"[[0.40664030180666166, 0.4513084114213143], [0...","{1, 2, 3}",foo6,"(1, 2, 3)"
7,True,/active/examples/fp_example/7.json,96.259690,"[[0.4192502702591062, 0.4240524465509987], [0....","{1, 2, 3}",foo7,"(1, 2, 3)"
8,True,/active/examples/fp_example/8.json,3.516826,"[[0.08427266973184566, 0.7325206981419501], [0...","{1, 2, 3}",foo8,"(1, 2, 3)"
9,True,/active/examples/fp_example/9.json,22.085252,"[[0.055019993340200135, 0.5232460707782919], [...","{1, 2, 3}",foo9,"(1, 2, 3)"


In [14]:
local._deep_print()

------------------------------- DATASET DATABASE -------------------------------
--------------------------------------------------------------------------------
User:
rows: 1
recent:
--------------------------------------------------------------------------------
Iota:
rows: 114
recent:
--------------------------------------------------------------------------------
SourceType:
rows: 2
recent:
--------------------------------------------------------------------------------
Source:
rows: 2
recent:
--------------------------------------------------------------------------------
FileSource:
rows: 2
recent:
--------------------------------------------------------------------------------
Dataset:
rows: 2
recent:
--------------------------------------------------------------------------------
IotaDatasetJunction:
rows: 160
recent:
--------------------------------------------------------------------------------
Algorithm:
rows: 1
recent:
------------------------------------------------------